# SAT example

In [1]:
from pref_voting.profiles import Profile
from itertools import combinations, permutations, product, chain
from pysat.solvers import Glucose3 # SAT solver


In [2]:

# powerset function needed to 
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))


**Theorem 13.1.** For $n = 3$ voters and $m > 3$ alternatives, no (resolute) voting rule satisﬁes both strategyproofness and the majority criterion.

In [3]:
def maj_winner(profiles, cand): 
    """Returns the profiles in which a strict majority of voters rank cand in first-place"""
    
    ps = list()
    for prof in profiles: 
        maj_size = prof.strict_maj_size()
        pl_scores = prof.plurality_scores()
        if pl_scores[cand] >= maj_size: 
            ps.append(prof)
    return ps


In [34]:
def find_possible_winners(profiles): 
    """
    Returns a list of tuples of consisting of a profile and a possible winning set. 
    
    Since we are considering resolute voting rules, each candidate can be a winner. 
    """
    
    possible_winners = list()
    
    for prof in profiles: 
        candidates = prof.candidates
        
        for w in candidates: 
            possible_winners.append((prof, w))
            
    return possible_winners


In [35]:
def generate_pws_dictionary(possible_winners):
    """
    Given a list of profile, candidate pairs, create a dictionary mapping each pair to a number. 
    """
    pws_dictionary = {}
    _id = 1
    
    for p,ws in possible_winners:
        pws_dictionary.update({(p, tuple(ws)): _id})
        _id += 1
    return pws_dictionary

In [36]:
def generate_function_formula(profiles, pws_dict): 
    """
    takes a list of all graphs 'graphs' and a dictionary of possible winners on those graphs 'wwt_dictionary'
    returns CNF clauses ensuring that only one winning set will be chosen per graph
    """
    C_func = list()
    for prof in profiles:     
        # get the list of all atomic formulas describing a possible winning set
        poss_ws = [pws_dict[(_p, ws)] for _p,ws in pws_dict.keys() if _p == prof]
        C_func.append(poss_ws) # one of the possible winning sets is the winner
        for pairs in combinations(poss_ws, 2):
            # add a clause saying that it is not the case that two possible winning sets are associated 
            # with a weighted tournament
            C_func.append([0 - pairs[0], 0 - pairs[1]])

    return C_func

In [37]:
def display_vm(model, pws_dict): 
    
    for i in model: 
        if i > 0: 
            for prof, ws in pws_dict: 
                if pws_dict[(prof, ws)] == i:
                    prof.display()
                    print(f"The winning set is: {'{' + ','.join(list(map(str,ws))) + '}'}")

In [38]:
# Sanity check: This should output True since it is always possible 
# to find some weighted tournament solution on a domain of weighted tournaments

num_cands = 3
num_voters = 3

candidates = range(3)

all_rankings = list(permutations(candidates))
all_profiles = [Profile(ps) for ps in product(all_rankings, repeat=3)]

possible_winners = find_possible_winners(all_profiles)
print("Done finding the possible winners.")

pws_dict = generate_pws_dictionary(find_possible_winners(all_profiles))
print("Done creating the encoding.")

C_func = generate_function_formula(all_profiles, pws_dict)
print("Done creating the function formula.\n\n")

sat_solver = Glucose3()
for cl in C_func:
    sat_solver.add_clause(cl)
print(f"Should be true expressing that there is a voting method on the domain: {sat_solver.solve()}\n")

# uncomment to see the model (list of literals that are true)
print(sat_solver.get_model())

display_vm(sat_solver.get_model(), pws_dict) 


Done finding the possible winners.
Done creating the encoding.
Done creating the function formula.


Should be true expressing that there is a voting method on the domain: True

[1, -2, -3, 4, -5, -6, 7, -8, -9, 10, -11, -12, 13, -14, -15, 16, -17, -18, 19, -20, -21, 22, -23, -24, 25, -26, -27, 28, -29, -30, 31, -32, -33, 34, -35, -36, 37, -38, -39, 40, -41, -42, 43, -44, -45, 46, -47, -48, 49, -50, -51, 52, -53, -54, 55, -56, -57, 58, -59, -60, 61, -62, -63, 64, -65, -66, 67, -68, -69, 70, -71, -72, 73, -74, -75, 76, -77, -78, 79, -80, -81, 82, -83, -84, 85, -86, -87, 88, -89, -90, 91, -92, -93, 94, -95, -96, 97, -98, -99, 100, -101, -102, 103, -104, -105, 106, -107, -108, 109, -110, -111, 112, -113, -114, 115, -116, -117, 118, -119, -120, 121, -122, -123, 124, -125, -126, 127, -128, -129, 130, -131, -132, 133, -134, -135, 136, -137, -138, 139, -140, -141, 142, -143, -144, 145, -146, -147, 148, -149, -150, 151, -152, -153, 154, -155, -156, 157, -158, -159, 160, -161, -162, 163, -164, 

In [39]:
def generate_maj_formula(cands, profiles, pws_dict): 
    """
    takes a list of all graphs 'graphs' and a dictionary of possible winners on those graphs 'wwt_dictionary'
    returns CNF clauses ensuring that only one winning set will be chosen per graph
    """
    C_maj = list()
    
    for c in cands: 
        for prof in maj_winner(profiles, c): 
            C_maj.append([pws_dict[(prof,(c,))]])
    return C_maj

In [57]:


C_maj_formula = generate_maj_formula(candidates, all_profiles, pws_dict)

sat_solver = Glucose3()

for cl in C_func + C_maj_formula:
    sat_solver.add_clause(cl)
print(f"Should be true expressing that there is a voting method on the domain: {sat_solver.solve()}\n")

# uncomment to see the model (list of literals that are true)
print(sat_solver.get_model())

display_vm(sat_solver.get_model(), pws_dict) 


Should be true expressing that there is a voting method on the domain: True

[1, -2, -3, 4, -5, -6, 7, -8, -9, 10, -11, -12, 13, -14, -15, 16, -17, -18, 19, -20, -21, 22, -23, -24, 25, -26, -27, 28, -29, -30, 31, -32, -33, 34, -35, -36, 37, -38, -39, 40, -41, -42, -43, 44, -45, -46, 47, -48, 49, -50, -51, 52, -53, -54, 55, -56, -57, 58, -59, -60, -61, 62, -63, -64, 65, -66, 67, -68, -69, 70, -71, -72, 73, -74, -75, 76, -77, -78, 79, -80, -81, 82, -83, -84, -85, -86, 87, -88, -89, 90, 91, -92, -93, 94, -95, -96, 97, -98, -99, 100, -101, -102, -103, -104, 105, -106, -107, 108, 109, -110, -111, 112, -113, -114, 115, -116, -117, 118, -119, -120, 121, -122, -123, 124, -125, -126, 127, -128, -129, 130, -131, -132, 133, -134, -135, 136, -137, -138, 139, -140, -141, 142, -143, -144, 145, -146, -147, 148, -149, -150, -151, 152, -153, -154, 155, -156, 157, -158, -159, 160, -161, -162, 163, -164, -165, 166, -167, -168, -169, 170, -171, -172, 173, -174, 175, -176, -177, 178, -179, -180, 181, -182,

In [53]:

def generate_strat_formula(voters, cands, profiles, pws_dict): 
    """
    takes a list of all graphs 'graphs' and a dictionary of possible winners on those graphs 'wwt_dictionary'
    returns CNF clauses ensuring that only one winning set will be chosen per graph
    """
    C_strat = list()
    
    for v in voters: 
        for prof in profiles: 
            for prof2 in profiles: 
                if all([prof.rankings[_v] == prof2.rankings[_v] for _v in voters if _v != v]):
                    for _p,ws in pws_dict.keys(): 
                        for _p2,ws2 in pws_dict.keys(): 
                            if _p == prof and _p2 == prof2 and prof.rankings[v].index(ws2[0]) < prof.rankings[v].index(ws[0]): 
                                C_strat.append([-pws_dict[(_p, ws)], -pws_dict[(_p2, ws2)]])
    return C_strat



CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 11 µs


In [54]:
%%time 

C_strat_formula = generate_strat_formula(range(3), candidates, all_profiles, pws_dict)
print("Finished generating the strategy-proof formula")



Finished generating the strategy-proof formula
CPU times: user 1min 29s, sys: 167 ms, total: 1min 29s
Wall time: 1min 29s


In [63]:
sat_solver = Glucose3()

for cl in C_func + C_maj_formula + C_strat_formula:
    sat_solver.add_clause(cl)
print(f"Is there a satisfying assignment: {sat_solver.solve()}\n")

# uncomment to see the model (list of literals that are true)
#print(sat_solver.get_model())

#display_vm(sat_solver.get_model(), pws_dict) 


Is there a satisfying assignment: False

None


TypeError: 'NoneType' object is not iterable